# Electricity Usage Analysis

In [1]:
for name in dir():
    if not name.startswith("_"):
        del globals()[name]

In [2]:
import sys
!{sys.executable} -m pip install hvplot

In [17]:
# Libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import hvplot.pandas

from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [5]:
df = pd.read_csv('df3.csv', header='infer')
df.set_index('date', inplace=True)
df['Hour'] = df['Hour'].astype(str)

In [6]:
print(df.shape)
df.head()

(16459, 29)


,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,v1,Weekday,Hour
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.20,48.900000,17.033333,45.53,6.600000,733.5,92.000000,7.000000,63.000000,5.3,13.275433,Mon,17
2016-01-11 17:10:00,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,21.79,48.863333,18.890000,45.56,6.483333,733.6,92.000000,6.666667,40.000000,5.2,18.606195,Mon,17
2016-01-11 17:20:00,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.20,48.730000,17.000000,45.50,6.366667,755.0,92.000000,6.333333,55.333333,5.1,28.642668,Mon,17
2016-01-11 17:30:00,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.10,48.590000,17.000000,45.40,6.250000,733.8,84.333333,6.000000,51.500000,5.0,45.410390,Mon,17
2016-01-11 17:40:00,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,34.927778,18.10,48.590000,17.000000,45.40,6.133333,733.9,84.333333,5.666667,47.666667,4.9,10.084097,Mon,17


In [7]:
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,v1
count,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.00000,16459.000000,16459.000000
mean,96.233064,3.955283,21.411526,40.169764,20.036250,40.440233,21.935220,39.375334,20.468408,39.095497,19.286999,51.174386,7.166662,59.619036,19.787542,35.457365,21.692460,43.134914,19.029985,41.566813,6.848526,754.794260,80.719657,4.120142,38.61759,3.335968,24.934725
std,102.028297,8.060528,1.389577,3.513932,1.875805,3.643092,1.668433,3.038812,1.807956,3.971781,1.548120,8.694324,5.296161,28.978911,1.760854,4.457492,1.717435,4.855274,1.674996,3.856741,4.901699,6.937394,14.110826,2.358126,10.59493,3.680742,14.517159
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.596667,17.200000,30.663333,15.100000,27.660000,15.330000,29.815000,-6.030000,1.000000,15.390000,23.200000,16.362222,29.600000,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.00000,-6.600000,0.005322
25%,50.000000,0.000000,20.790000,37.900000,18.890000,38.400000,20.823333,37.290000,19.390000,36.290000,18.200000,46.090000,3.933333,40.725000,18.700000,32.500000,20.790000,39.568036,18.000000,38.976500,3.663889,751.100000,73.000000,2.333333,32.50000,1.166667,12.425313
50%,60.000000,0.000000,21.390000,39.626667,19.760000,40.590000,21.890000,38.730000,20.390000,38.500000,19.260000,49.400000,6.656667,62.261667,19.600000,34.927778,21.790000,42.600000,18.890000,41.000000,6.400000,755.000000,84.333333,3.833333,40.00000,3.200000,24.774977
75%,100.000000,0.000000,22.100000,42.433333,20.790000,42.826667,22.790000,41.290000,21.390000,41.465000,20.200000,53.334000,9.726667,84.945000,20.926667,37.905409,22.730000,46.060000,20.251429,43.760000,9.600000,759.266667,91.166667,5.333333,40.00000,5.486111,37.583769
max,1080.000000,70.000000,25.700000,63.360000,29.856667,56.026667,27.600000,50.163333,26.200000,51.090000,25.745000,96.321667,28.236000,99.900000,25.390000,51.400000,27.230000,58.780000,24.200000,53.326667,26.100000,772.300000,100.000000,14.000000,66.00000,15.500000,49.996530


In [203]:
# some testing ...
if 0:
    x = df3['Appliances'] - df3['Appliances'].min() + 1
    print(x[0:10].tolist())

    xbox, opt_lambda = boxcox(x)
    print(xbox[0:10].tolist())
    print(opt_lambda)

    xinvbox = inv_boxcox(xbox, opt_lambda)
    print(xinvbox[0:10].tolist())

[51.0, 51.0, 41.0, 41.0, 51.0, 41.0, 51.0, 51.0, 51.0, 61.0]
[2.646922942509263, 2.646922942509263, 2.5515385251450624, 2.5515385251450624, 2.646922942509263, 2.5515385251450624, 2.646922942509263, 2.646922942509263, 2.646922942509263, 2.7218760785681684]
-0.21655783519823987
[51.00000000000002, 51.00000000000002, 41.00000000000001, 41.00000000000001, 51.00000000000002, 41.00000000000001, 51.00000000000002, 51.00000000000002, 51.00000000000002, 60.99999999999995]


In [8]:
# df
def get_filtered_df(df, excluded_cols):
    features = [col for col in df.columns if col not in excluded_cols]
    X = df[features].copy()
    y = df['Appliances']
    return (X, y)

In [9]:
def log_transform(v, subtract_min=1, add_one=0):
    vt = v
    if (subtract_min):
        vt = v - v.min() + 1
    elif (add_one):
        vt = v + 1
    
    return np.log(vt)

In [10]:
# log transformation of selected features in a dataaframe
def log_transform_features(X, features, subtract_min=1, add_one=0):
    Xt = X.copy()
    for col in features:
        vt = log_transform(X[col], subtract_min, add_one)
        Xt[col] = vt

    return (Xt)

In [11]:
def boxcox_transform(v, subtract_min=1):
    v1 = v
    vmin = v.min()
    if (subtract_min):
        v1 = v - vmin + 1
    vt, opt_lambda = boxcox(v1)
    return (vt, vmin, opt_lambda)

In [12]:
# Box-Cox transformation of selected features in a dataaframe
def boxcox_transform_features(X, features):
    Xt = X.copy()
    opt_lambda_dict = {}
    for col in features:
        vt, vmin, opt_lambda = boxcox_transform(X[col])
        Xt[col] = vt
        opt_lambda_dict[col] = opt_lambda

    return (Xt, opt_lambda_dict)

In [13]:
def one_hot_encoding(X, cat_col_list):
    df_cat = X[cat_col_list]

    encoding_cat = OneHotEncoder()
    encoding_cat.fit(df_cat)
    df_cat_enc = pd.DataFrame(
        encoding_cat.transform(df_cat).toarray(), columns=encoding_cat.get_feature_names()
    )

    df_cat_enc.set_index(df_cat.index, inplace=True)
    Xt = pd.merge(X, df_cat_enc, how='left', left_index=True, right_index=True)
    Xt = Xt.drop(cat_col_list, axis=1)
    
    return (Xt)

In [14]:
#scoring = make_scorer(r2_score)
scoring = make_scorer(root_mean_squared_error, greater_is_better=False)

In [15]:
#-------------------------------------------------------------------------------
# Execute the cross-validation for all parameter combinations and plots the
# evaluation metrics. At each iteration of the cross validation process, the 
# model is evaluated using all the below metrics but the optimal model is 
# determined using the AUC score (as this minimizes both false positives &
# false negatives).
#-------------------------------------------------------------------------------
def build_optimal_model(model_type, model, model_parameter_grid, X, y, scoring):
    grid_object = GridSearchCV(model, param_grid=model_parameter_grid, cv=5, scoring=scoring, refit=True, return_train_score=True, n_jobs=-1)
    print('Performing grid search to determine the optimal parameters ...')
    grid_object.fit(X, y)
    grid_search_output = pd.DataFrame(grid_object.cv_results_)

    optimal_model = grid_object.best_estimator_
    optimal_model = optimal_model.fit(X, y)

    return grid_object, grid_search_output, optimal_model

In [22]:
# Decision Tree setup
model_type = 'Xgboost'
model = XGBRegressor(
    random_state=1, n_estimators=100, objective='reg:squarederror', booster='gbtree', max_depth=5
)
MAX_HEIGHT = 20
model_parameter_grid = {
         'reg_alpha': [0.05, 0.1, 0.2, 0.3],
         'reg_lambda': [0.05, 0.1, 0.2, 0.3]
        }

In [23]:
%%time

# 1. log transform of Appliances variable

excluded_cols = ['Appliances', 'Weekday', 'Hour']
X, y = get_filtered_df(df, excluded_cols)

Xt = X
yt = log_transform(y, subtract_min=0, add_one=0)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


CPU times: user 10.6 s, sys: 342 ms, total: 11 s
Wall time: 49.9 s


In [24]:
#print(grid_object)
# grid_search_output.head()
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

0.6713656396048717
{'reg_alpha': 0.05, 'reg_lambda': 0.2}


,feature,importance
0,lights,0.198926
21,RH_out,0.077561
16,RH_8,0.053609
17,T9,0.052456
3,T2,0.047719
2,RH_1,0.039954
9,T5,0.039206
6,RH_3,0.034515
18,RH_9,0.034384
15,T8,0.031470


In [25]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.42694084240124497
79.45989458137981
0.580003498721406
0.3934297833542165


In [26]:
%%time

# 2. Box-Cox transformation of features

excluded_cols = ['Appliances', 'Weekday', 'Hour']
X, y = get_filtered_df(df, excluded_cols)

Xt, opt_lambda_dict = boxcox_transform_features(X, X.columns.tolist())
yt, _, opt_lambda = boxcox_transform(y, subtract_min=0)
opt_lambda_dict['Appliances'] = opt_lambda # Appliances > 0

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...
CPU times: user 10.8 s, sys: 93.1 ms, total: 10.9 s
Wall time: 49.7 s


In [27]:
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

0.05948550713862301
{'reg_alpha': 0.3, 'reg_lambda': 0.05}


,feature,importance
0,lights,0.362430
17,T9,0.079205
3,T2,0.064827
16,RH_8,0.042427
21,RH_out,0.033313
5,T3,0.031154
2,RH_1,0.029436
6,RH_3,0.027423
15,T8,0.027225
18,RH_9,0.025228


In [28]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(inv_boxcox(predicted, opt_lambda_dict['Appliances']), y))

print(r2_score(yt, predicted))
print(r2_score(y, inv_boxcox(predicted, opt_lambda_dict['Appliances'])))

0.043836326796503275
90.19743585160536
0.5254349700810284
0.21841989483853064


In [29]:
%%time

# 3. One-hot encoding of Weekday & Hour features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

Xt = one_hot_encoding(X, ['Weekday', 'Hour'])
yt = log_transform(y, subtract_min=0, add_one=0)

#print(Xt.shape)
#Xt.head()

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


CPU times: user 16.4 s, sys: 609 ms, total: 17 s
Wall time: 1min 30s


In [30]:
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

0.6079400482677418
{'reg_alpha': 0.2, 'reg_lambda': 0.3}


,feature,importance
0,lights,0.066094
43,x1_18,0.063832
44,x1_19,0.044959
50,x1_3,0.039184
42,x1_17,0.038471
33,x1_0,0.037403
49,x1_23,0.036789
45,x1_2,0.036383
51,x1_4,0.035612
34,x1_1,0.034364


In [31]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.42381970210410186
80.80604918590062
0.5861218005622599
0.3727035062909164


In [32]:
%%time

# 4. One-hot encoding of Weekday & Hour features followed by Box-Cox transformation of numerical features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1, opt_lambda_dict = boxcox_transform_features(X, features_to_transform)
yt, _, opt_lambda = boxcox_transform(y, subtract_min=0)
opt_lambda_dict['Appliances'] = opt_lambda # Appliances > 0

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 16.9 s, sys: 358 ms, total: 17.2 s
Wall time: 1min 25s


In [33]:
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

0.05492081143292752
{'reg_alpha': 0.2, 'reg_lambda': 0.2}


,feature,importance
0,lights,0.140655
43,x1_18,0.052562
44,x1_19,0.048144
46,x1_20,0.043542
50,x1_3,0.036587
34,x1_1,0.035526
42,x1_17,0.034961
45,x1_2,0.033761
51,x1_4,0.032869
33,x1_0,0.032259


In [35]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(inv_boxcox(predicted, opt_lambda_dict['Appliances']), y))

print(r2_score(yt, predicted))
print(r2_score(y, inv_boxcox(predicted, opt_lambda_dict['Appliances'])))

0.04197008048358709
88.98995902381836
0.5649822030039011
0.23920591640906308


In [36]:
%%time

# 5. One-hot encoding of Weekday & Hour features followed by log transformation of numerical features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1 = log_transform_features(X, features_to_transform, subtract_min=1, add_one=0)
yt = log_transform(y, subtract_min=0, add_one=0)

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


CPU times: user 16.6 s, sys: 503 ms, total: 17.1 s
Wall time: 1min 31s


In [37]:
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

0.607962554775313
{'reg_alpha': 0.2, 'reg_lambda': 0.3}


,feature,importance
0,lights,0.066094
43,x1_18,0.063832
44,x1_19,0.044959
50,x1_3,0.039184
42,x1_17,0.038471
33,x1_0,0.037403
49,x1_23,0.036789
45,x1_2,0.036383
51,x1_4,0.035612
34,x1_1,0.034364


In [38]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.42381970210410186
80.80604918590062
0.5861218005622599
0.3727035062909164


# Experiments on scaled data

In [39]:
df = pd.read_csv('df3_scaled_range.csv', header='infer')
df.set_index('date', inplace=True)
df['Hour'] = df['Hour'].astype(str)

print(df.shape)
df.head()

(16459, 29)


,Appliances,Weekday,Hour,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,v1
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60.0,Mon,17,0.428571,0.347924,0.566187,0.225345,0.682849,0.249038,0.721368,0.351351,0.764262,0.176348,0.381691,0.381039,0.841827,0.181000,0.653428,0.169103,0.661412,0.230218,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449
2016-01-11 17:10:00,60.0,Mon,17,0.428571,0.347924,0.541326,0.225345,0.680944,0.249038,0.724444,0.351351,0.782437,0.176348,0.381691,0.375396,0.839872,0.181000,0.651064,0.499438,0.660155,0.429646,0.678532,0.369239,0.100000,0.894737,0.476190,0.600000,0.533937,0.372083
2016-01-11 17:20:00,50.0,Mon,17,0.428571,0.347924,0.530502,0.225345,0.678239,0.249038,0.731795,0.344745,0.778062,0.176348,0.380037,0.367420,0.830704,0.181000,0.646572,0.169103,0.655586,0.226638,0.676049,0.365488,0.597674,0.894737,0.452381,0.835897,0.529412,0.572848
2016-01-11 17:30:00,50.0,Mon,17,0.571429,0.347924,0.524080,0.225345,0.677204,0.249038,0.735214,0.341441,0.770949,0.176348,0.380037,0.363723,0.833401,0.174333,0.641489,0.159902,0.650788,0.226638,0.671909,0.361736,0.104651,0.793860,0.428571,0.776923,0.524887,0.908261
2016-01-11 17:40:00,60.0,Mon,17,0.571429,0.347924,0.531419,0.225345,0.675510,0.249038,0.735214,0.341441,0.762697,0.179549,0.380037,0.361777,0.848264,0.181000,0.415879,0.159902,0.650788,0.226638,0.671909,0.357985,0.106977,0.793860,0.404762,0.717949,0.520362,0.201611


In [40]:
%%time

# 3. One-hot encoding of Weekday & Hour features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

Xt = one_hot_encoding(X, ['Weekday', 'Hour'])
yt = log_transform(y, subtract_min=0, add_one=0)

#print(Xt.shape)
#Xt.head()

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.6079540213082028
{'reg_alpha': 0.2, 'reg_lambda': 0.3}
CPU times: user 16.6 s, sys: 590 ms, total: 17.1 s
Wall time: 1min 30s


,feature,importance
0,lights,0.066094
43,x1_18,0.063832
44,x1_19,0.044959
50,x1_3,0.039184
42,x1_17,0.038471
33,x1_0,0.037403
49,x1_23,0.036789
45,x1_2,0.036383
51,x1_4,0.035612
34,x1_1,0.034364


In [41]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.42381970210410186
80.80604918590062
0.5861218005622599
0.3727035062909164


In [42]:
%%time

# 4. One-hot encoding of Weekday & Hour features followed by Box-Cox transformation of numerical features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1, opt_lambda_dict = boxcox_transform_features(X, features_to_transform)
yt, _, opt_lambda = boxcox_transform(y, subtract_min=0)
opt_lambda_dict['Appliances'] = opt_lambda # Appliances > 0

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.054919893615577954
{'reg_alpha': 0.2, 'reg_lambda': 0.2}
CPU times: user 19.4 s, sys: 420 ms, total: 19.8 s
Wall time: 1min 42s


,feature,importance
0,lights,0.140655
43,x1_18,0.052562
44,x1_19,0.048144
46,x1_20,0.043542
50,x1_3,0.036587
34,x1_1,0.035526
42,x1_17,0.034961
45,x1_2,0.033761
51,x1_4,0.032869
33,x1_0,0.032259


In [44]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(inv_boxcox(predicted, opt_lambda_dict['Appliances']), y))

print(r2_score(yt, predicted))
print(r2_score(y, inv_boxcox(predicted, opt_lambda_dict['Appliances'])))

0.04197008048358709
88.98995902381836
0.5649822030039011
0.23920591640906308


In [49]:
%%time

# 5. One-hot encoding of Weekday & Hour features followed by log transformation of numerical features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1 = log_transform_features(X, features_to_transform, subtract_min=1, add_one=0)
yt = log_transform(y, subtract_min=0, add_one=0)

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.6079723375711229
{'reg_alpha': 0.2, 'reg_lambda': 0.3}
CPU times: user 16 s, sys: 552 ms, total: 16.6 s
Wall time: 1min 28s


,feature,importance
0,lights,0.066094
43,x1_18,0.063832
44,x1_19,0.044959
50,x1_3,0.039184
42,x1_17,0.038471
33,x1_0,0.037403
49,x1_23,0.036789
45,x1_2,0.036383
51,x1_4,0.035612
34,x1_1,0.034364


In [50]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.42381970210410186
80.80604918590062
0.5861218005622599
0.3727035062909164


# Experiments on standardized data

In [51]:
df = pd.read_csv('df3_scaled_std.csv', header='infer')
df.set_index('date', inplace=True)
df['Hour'] = df['Hour'].astype(str)

In [52]:
%%time

# 3. One-hot encoding of Weekday & Hour features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

Xt = one_hot_encoding(X, ['Weekday', 'Hour'])
yt = log_transform(y, subtract_min=0, add_one=0)

#print(Xt.shape)
#Xt.head()

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.6079616864507625
{'reg_alpha': 0.2, 'reg_lambda': 0.3}
CPU times: user 20.9 s, sys: 491 ms, total: 21.4 s
Wall time: 1min 38s


,feature,importance
0,lights,0.066094
43,x1_18,0.063832
44,x1_19,0.044959
50,x1_3,0.039184
42,x1_17,0.038471
33,x1_0,0.037403
49,x1_23,0.036789
45,x1_2,0.036383
51,x1_4,0.035612
34,x1_1,0.034364


In [53]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.42381970210410186
80.80604918590062
0.5861218005622599
0.3727035062909164


In [54]:
%%time

# 4. One-hot encoding of Weekday & Hour features followed by Box-Cox transformation of numerical features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1, opt_lambda_dict = boxcox_transform_features(X, features_to_transform)
yt, _, opt_lambda = boxcox_transform(y, subtract_min=0)
opt_lambda_dict['Appliances'] = opt_lambda # Appliances > 0

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.05492021367070194
{'reg_alpha': 0.2, 'reg_lambda': 0.2}
CPU times: user 18.2 s, sys: 546 ms, total: 18.7 s
Wall time: 1min 39s


,feature,importance
0,lights,0.140655
43,x1_18,0.052562
44,x1_19,0.048144
46,x1_20,0.043542
50,x1_3,0.036587
34,x1_1,0.035526
42,x1_17,0.034961
45,x1_2,0.033761
51,x1_4,0.032869
33,x1_0,0.032259


In [55]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(inv_boxcox(predicted, opt_lambda_dict['Appliances']), y))

print(r2_score(yt, predicted))
print(r2_score(y, inv_boxcox(predicted, opt_lambda_dict['Appliances'])))

0.04197008048358709
88.98995902381836
0.5649822030039011
0.23920591640906308


In [56]:
%%time

# 5. One-hot encoding of Weekday & Hour features followed by log transformation of numerical features

excluded_cols = ['Appliances']
X, y = get_filtered_df(df, excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1 = log_transform_features(X, features_to_transform, subtract_min=1, add_one=0)
yt = log_transform(y, subtract_min=0, add_one=0)

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.607959361961047
{'reg_alpha': 0.2, 'reg_lambda': 0.3}
CPU times: user 18.8 s, sys: 505 ms, total: 19.3 s
Wall time: 1min 37s


,feature,importance
0,lights,0.066094
43,x1_18,0.063832
44,x1_19,0.044959
50,x1_3,0.039184
42,x1_17,0.038471
33,x1_0,0.037403
49,x1_23,0.036789
45,x1_2,0.036383
51,x1_4,0.035612
34,x1_1,0.034364


In [57]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.42381970210410186
80.80604918590062
0.5861218005622599
0.3727035062909164
